# Паттерн "Абстрактная фабрика"

Импортируем необходимые классы и методы

In [12]:
from abc import ABC, abstractmethod

## 1. Объявим абстрактый класс фабрики

Обявим методы, которые позволят создать персонажа, а также оружие и заклинание для него.

In [13]:
class HeroFactory(ABC):
    @abstractmethod
    def create_hero(self, name):
        pass
    
    @abstractmethod
    def create_weapon(self):
        pass
    
    @abstractmethod
    def create_spell(self):
        pass

## 2. Определим конкретные фабрики 

Оределим конкретные фабрики и необходимые классы, для каждого из классов персонажей

In [60]:
class WarriorFactory(HeroFactory):
    def create_hero(self, name):
        return Warrior(name)
    
    def create_weapon(self):
        return Claymore()
    
    def create_spell(self):
        return Power()

class Warrior:
    def __init__(self, name):
        self.name = name
        self.weapon = None
        self.armor = None
        self.spell = None
        
    def add_weapon(self, weapon):
        self.weapon = weapon
        
    def add_spell(self, spell):
        self.spell = spell
        
    def hit(self):
        print(f"Warrior {self.name} hits with {self.weapon.hit()}")
        self.weapon.hit()
        
    def cast(self):
        print(f"Warrior {self.name} casts {self.spell.cast()}")
        self.spell.cast()
        

class Claymore:
    def hit(self):
        return "Claymore"


class Power:
    def cast(self):
        return "Power"


class MageFactory(HeroFactory):
    def create_hero(self, name):
        return Mage(name)
    
    def create_weapon(self):
        return Staff()
    
    def create_spell(self):
        return Fireball()


class Mage:
    def __init__(self, name):
        self.name = name
        self.weapon = None
        self.armor = None
        self.spell = None
        
    def add_weapon(self, weapon):
        self.weapon = weapon
        
    def add_spell(self, spell):
        self.spell = spell
        
    def hit(self):
        print(f"Mage {self.name} hits with {self.weapon.hit()}")
        self.weapon.hit()
        
    def cast(self):
        print(f"Mage {self.name} casts {self.spell.cast()}")
        self.spell.cast()

        
class Staff:
    def hit(self):
        return "Staff"
    
    
class Fireball:
    def cast(self):
        return "Fireball"


class AssassinFactory(HeroFactory):
    def create_hero(self, name):
        return Assassin(name)
    
    def create_weapon(self):
        return Dagger()
    
    def create_spell(self):
        return Invisibility()


class Assassin:
    def __init__(self, name):
        self.name = name
        self.weapon = None
        self.armor = None
        self.spell = None
        
    def add_weapon(self, weapon):
        self.weapon = weapon
        
    def add_spell(self, spell):
        self.spell = spell
        
    def hit(self):
        print(f"Assassin {self.name} hits with {self.weapon.hit()}")
        self.weapon.hit()
        
    def cast(self):
        print(f"Assassin {self.name} casts {self.spell.cast()}")


class Dagger:
    def hit(self):
        return "Dagger"
    

class Invisibility:
    def cast(self):
        return "Invisibility"

## 3. Определим функцию, создающую персонажей

Определим функцию, зависящую от фабрики. Данная функция будет создавать прсонажа и его экипировку в зависимости от фабрики, которая будет передаваться в качестве аргумента.

In [43]:
import yaml

hero_yaml = '''
--- !Character # указывает, что хранящаяся ниже структура относиться к классу Character
first: # реквизит first
    assassin # значение реквизита
second: # реквизит second
    Pavlyk # значение реквизита
'''

hero = yaml.load(hero_yaml, Loader=yaml.Loader)
hero.__dict__


Character new


{'first': 'assassin', 'second': 'Pavlyk'}

In [46]:
# В момент загрузки yaml'a автоматически понимает что это класс Character
# создаем объект Character с реквизитами factory и name
# т.к. реквизит factory не простого типа то ему написал отдельный обработчки factory_constructor

hero_yaml = '''
--- !Character # указывает, что хранящаяся ниже структура относиться к классу Character
factory: # реквизит factory
    !factory assassin # значение реквизита имеет тип factory и данные assassin
name: # реквизит name
    Pavlyk # значение реквизита
'''

In [73]:
def factory_constructor(loader, node):
    # print('factory_constructor', loader, node) # loader это наш загрузчик где правила создания # node это наши данные где остановились
    data = loader.construct_scalar(node) # construct_scalar checks that the given node is a scalar and returns its value.
    if data == "assassin":
        return AssassinFactory
    elif data == "mage":
        return MageFactory
    else:
        return WarriorFactory

class Character(yaml.YAMLObject):
    yaml_tag = "!Character"
    # def __new__(cls):
    #     print('Character new')
    #     return super().__new__(cls)

    # @classmethod # Если переопределить класс from_yaml то можно вернуть что угодно.после load(), иначе возвращается класс Character
    # def from_yaml(Class, loader, node):
    #     return 'hello world'

    def create_hero(self):
        print(self.factory)
        hero = self.factory().create_hero(self.name)
        
        # weapon = factory.create_weapon()
        # ability = factory.create_spell()
        
        # hero.add_weapon(weapon)
        # hero.add_spell(ability)
        
        return hero

In [74]:
loader = yaml.Loader
loader.add_constructor("!factory", factory_constructor)

In [75]:
# hero = yaml.load(hero_yaml) # Старый синтаксис
character = yaml.load(hero_yaml, Loader=loader)
print(type(character))
print(character)
print(character.__dict__)

hero = character.create_hero()
print(hero)
# hero.hit()

<class '__main__.Character'>
{'factory': <class '__main__.AssassinFactory'>, 'name': 'Pavlyk'}
<class '__main__.AssassinFactory'>


## 4. Попробуем создать персонажей различных классов

Попробуем создать персонажей различных классов, передавая функции назличные фабрики.

In [5]:
factory = AssassinFactory()
player = create_hero(factory)
player.cast()
player.hit()

Assassin Nagibator casts Invisibility
Assassin Nagibator hits with Dagger


In [6]:
factory = MageFactory()
player = create_hero(factory)
player.cast()
player.hit()

Mage Nagibator casts Fireball
Mage Nagibator hits with Staff
